In [2]:
# imports
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')
print(FOURSQUARE_KEY)
# confirm API key has been imported successfully from Terminal

fsq3F074rL3S6i0kGeOeRWB1KhL1WLyqB4dyTOG72DKFm4A=


In [10]:
def get_venues_fs(latitude, longitude, radius, api_key, categories, limit):
    """
    Get amenities and POIs from Foursquare API call
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query (imported in line above)
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        response: response object from the requests library.
    """
    url = "https://api.foursquare.com/v3/places/search"
    
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }
    
    params = {
        "ll": f"{latitude},{longitude}",
        "radius": radius,
        "categories": categories,
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200: # 200 is success
        return response.json()
    else:
        response.raise_for_status()
        

In [26]:
# testing
res = get_venues_fs(latitude=43.657763, longitude=-79.389165, radius=750, api_key=FOURSQUARE_KEY, categories=13003, limit=50)

print(len(res['results']))

print(res['results'][2]['name']) # this is how you access the name, we could loop through [i] to get names

    
# print(res['results'])

26
Prenup Pub


**Notes on the test API Call and justification of parameters:**
- Downtown Toronto is very densely packed with bike stations and the amenities I will be searching for. The default radius is 5km which is too large to capture the relationship between bike/dock availability and amenities. The appropriate range would be either **1km or even 750m**
- Foursquare Category IDs of interest are: 10035 (Performing Arts Venue), 12013 (College and University amenities), 13003 (Bars), 13065 (Restaurants)
- Other candidates: 16000 (landmarks & outdoors), 17057 (Food & Beverate Retail), 18000 (Sports & Recreaction)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating